In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2


import event_tools.tools as tl 
import visualization as vz
import event_tools.e2v as e2v

In [3]:
events = tl.load_event_data('./event_csv/split_data/class7/user02_lab.csv')  # Replace with your actual file path


In [3]:
# image definition/dvs image pixels 
height, width = 100, 200 
row, col = 100, 200
img = np.zeros((row, col), dtype=np.uint32)

In [4]:
horizontal_neurons = np.zeros_like(img[0, :]).reshape((1, -1))
vertical_neurons = np.zeros_like(img[:, 0]).reshape((-1, 1))

In [5]:
horizontal_neurons.shape, vertical_neurons.shape

((1, 200), (100, 1))

In [6]:
weight_input_horizontal_hidden = np.ones((1, row))
weight_input_vertical_hidden = np.ones((col, 1))

In [7]:
np.dot(weight_input_horizontal_hidden, img).shape

(1, 200)

In [8]:
np.dot(img, weight_input_vertical_hidden).shape

(100, 1)

In [9]:
events.shape

(440155, 3)

In [10]:
width=128
height=128

horizontal_member_potential = np.zeros(width) 
horizontal_neurons = np.zeros(width).reshape((1, -1))


decay = .01
random_wt_2d = lambda _row, _col: np.round(np.random.rand(_row, _col), 3)
weight_input_horizontal_hidden = random_wt_2d(1, height) # np.ones((1, height)) * weight_scale_1
weight_input_horizontal_hidden_1 = random_wt_2d(height, height//2) # np.ones((height, height//2)) * weight_scale_1

peak = 255 
spike = 255 
threshold = 0.8 


In [11]:
# check the correlation 
spikes_collection = []
for idx, frame_image in enumerate(
    e2v.get_event_frame(events, 
        decay=0.009, )):
    # spiking of the neurons
    # u[t+1] = Beta * u[t] + W * X[t+1] - R[t]
    # print(idx)
    horizontal_member_potential = (decay * horizontal_member_potential) +  np.dot(weight_input_horizontal_hidden, frame_image)
    # generating spikes 
    horizontal_neurons[horizontal_member_potential > (spike * threshold)] = 255 
    horizontal_neurons[horizontal_member_potential <= (spike * threshold)] = 0  
    # reset the membrane potential 
    horizontal_member_potential[horizontal_member_potential > (spike * threshold)] = 0 #        
    # prevent neurons to go to negative
    horizontal_member_potential[horizontal_member_potential < 0] = 0
    
    spikes_collection.append(horizontal_neurons.copy())
    cv2.imshow('tes', horizontal_neurons) 
    if(cv2.waitKey(1) == ord('q')):
        break 
cv2.destroyAllWindows()

In [47]:
frames_num = -1
num = 1
x = spikes_collection[:frames_num: num]
x = np.array(x).reshape(-1, 128)

In [48]:
x.shape

(2615, 128)

In [50]:
cv2.imshow('tes', x) 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
import torch 

In [52]:
x = torch.from_numpy(x) 

In [42]:
x = x[:64, :]
x.shape

torch.Size([64, 128])

In [53]:
x = torch.cdist(x, x) ** 2 
x = -x / 13
x = x.softmax(dim=-1)

In [57]:
# binary similarity matrix 
spike_trains = x 
similarity_matrix = spike_trains @ spike_trains.T 
print("Dot Prodct similarity matrix: \n ", similarity_matrix.shape)

Dot Prodct similarity matrix: 
  torch.Size([2615, 2615])


In [58]:
x.shape

torch.Size([2615, 2615])

In [59]:
def plot_heatmap(dist: np.ndarray, log_scale: bool = False) -> np.ndarray:
    """Plot the temporal self-similarity matrix into an OpenCV image."""
    np.fill_diagonal(dist, np.nan)
    if log_scale:
        dist = np.log(1 + dist)
    dist = -dist # Invert the distance
    zmin, zmax = np.nanmin(dist), np.nanmax(dist)
    heatmap = (dist - zmin) / (zmax - zmin) # Normalize into [0, 1]
    heatmap = np.nan_to_num(heatmap, nan=1)
    heatmap = np.clip(heatmap * 255, 0, 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_VIRIDIS)
    return heatmap

In [56]:
tsm_img = plot_heatmap(x.numpy(), log_scale=True)
cv2.imshow('tsm', tsm_img) 
cv2.waitKey(0)
cv2.destroyAllWindows()

Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


### extract data

In [ ]:
import os 
import glob 
import event_tools.tools as tl 
import event_tools.e2v as e2v 

In [ ]:
file_names = glob.glob('./event_csv/split_data/class?/*.csv')
for file_name in sorted(file_names):
    print("[*] ---- Processing file: ", file_name)
    event_class, event_user = file_name.split('/')[-2:]
    event_user = event_user.split('.')[0] 
    # main(event_class, event_user,
    #     out_csv_file = "artificial_encoder_replaced_output.csv")
    os.makedirs(f'./output/{event_class}', exist_ok=True)  
    out_file_path = f'./output/{event_class}/{event_user}.mp4'
    # load event camera data 
    events = tl.load_event_data(file_name)  # Replace with your actual file path
    e2v.save_event_as_video(events, out_file_path, frame_per_second=1/60)

In [4]:
events.shape


(440155, 3)

In [6]:
events[1:,1] - events[:-1,1]

array([ -7.,  11., -13., ...,   4.,  -4.,  16.])

In [7]:
diffs = np.abs(np.diff(events[:, 1]))
mask = np.concatenate(([True], diffs <= 5))
events[:,1][mask]

diffs = np.abs(np.diff(events[:, 1]))
mask = np.concatenate(([True], diffs <= 5))
events[:,1][mask]

array([70., 58., 60., ..., 60., 64., 60.])

In [10]:
a = []
p = events[0,1]
a.append(p)
for i in events[1:,1]:
    if not tl.is_with_in_manhattan_distance(p, 0, i, 0):
        p = i 
        continue
    print(i)  
    a.append(i)


58.0
60.0
56.0
56.0
55.0
60.0
60.0
55.0
66.0
58.0
66.0
64.0
58.0
64.0
62.0
73.0
67.0
60.0
61.0
55.0
61.0
58.0
59.0
59.0
60.0
65.0
58.0
59.0
65.0
61.0
75.0
72.0
68.0
83.0
80.0
66.0
64.0
58.0
57.0
61.0
60.0
59.0
58.0
80.0
65.0
68.0
69.0
65.0
58.0
61.0
61.0
66.0
60.0
60.0
73.0
73.0
71.0
62.0
61.0
54.0
57.0
54.0
72.0
69.0
54.0
58.0
60.0
60.0
54.0
53.0
63.0
63.0
57.0
57.0
66.0
74.0
67.0
61.0
78.0
57.0
58.0
60.0
63.0
55.0
62.0
54.0
56.0
60.0
60.0
56.0
54.0
55.0
60.0
66.0
58.0
54.0
54.0
65.0
68.0
57.0
52.0
58.0
63.0
57.0
59.0
57.0
76.0
82.0
62.0
54.0
61.0
59.0
81.0
79.0
79.0
55.0
57.0
57.0
61.0
60.0
57.0
81.0
73.0
78.0
74.0
76.0
81.0
67.0
55.0
65.0
61.0
60.0
62.0
55.0
59.0
57.0
58.0
54.0
54.0
79.0
58.0
58.0
71.0
72.0
81.0
78.0
67.0
65.0
70.0
52.0
66.0
67.0
59.0
58.0
56.0
67.0
53.0
57.0
68.0
72.0
81.0
76.0
57.0
60.0
62.0
83.0
62.0
63.0
59.0
64.0
68.0
60.0
63.0
59.0
63.0
54.0
57.0
53.0
64.0
55.0
53.0
55.0
61.0
58.0
56.0
54.0
72.0
71.0
63.0
66.0
73.0
74.0
77.0
82.0
82.0
64.0
71.0
57.0
57.0
60.0


In [11]:
a=np.array(a)  

In [13]:
a == events[:,1][mask]


ValueError: operands could not be broadcast together with shapes (172098,) (171911,) 

In [15]:
len(events[:,1])

440155

In [16]:
mask

array([ True, False, False, ...,  True,  True, False])